In [11]:
from ngraph.explorer import NetworkExplorer
from ngraph.scenario import Scenario

In [12]:
scenario_yaml = """
blueprints:
  bb_fabric:
    groups:
      t2:
        node_count: 4                 # always on
        name_template: t2-{node_num}

      t1:
        node_count: 16                # will be enabled in chunks
        name_template: t1-{node_num}

    adjacency:                         # full mesh, 2 parallel links
      - source: /t1
        target: /t2
        pattern: mesh
        link_count: 2
        link_params:
          capacity: 200
          cost: 1

network:
  name: "BB_Fabric"
  version: 1.0

  groups:
    bb_fabric:
      use_blueprint: bb_fabric

  # disable every T1 at load-time; workflow will enable them in batches
  node_overrides:
    - path: ^bb_fabric/t1/.+
      disabled: true

workflow:
  - step_type: EnableNodes
    path: ^bb_fabric/t1/.+
    count: 4                            # enable first group of T1s
    order: name

  - step_type: DistributeExternalConnectivity
    remote_prefix: remote/
    remote_locations:
      - LOC1
    attachment_path: ^bb_fabric/t1/.+    # enabled T1 nodes
    stripe_width: 2
    capacity: 800
    cost: 1

  - step_type: DistributeExternalConnectivity
    remote_prefix: remote/
    remote_locations:
      - LOC1
    attachment_path: ^bb_fabric/t1/.+    # enabled T1 nodes
    stripe_width: 2
    capacity: 800
    cost: 1
"""
scenario = Scenario.from_yaml(scenario_yaml)
network = scenario.network
explorer = NetworkExplorer.explore_network(network, scenario.components_library)
explorer.print_tree(include_disabled=False, detailed=False, skip_leaves=True)

- root | Nodes=20, Links=128, Cost=0.0, Power=0.0
  - bb_fabric | Nodes=20, Links=128, Cost=0.0, Power=0.0
    - t2 | Nodes=4, Links=128, Cost=0.0, Power=0.0
    - t1 | Nodes=16, Links=128, Cost=0.0, Power=0.0


In [13]:
network.nodes["bb_fabric/t1/t1-4"]

Node(name='bb_fabric/t1/t1-4', disabled=True, risk_groups=set(), attrs={'type': 'node'})

In [14]:
scenario.run()

In [15]:
explorer = NetworkExplorer.explore_network(network, scenario.components_library)
explorer.print_tree(skip_leaves=True, detailed=False)

- root | Nodes=21, Links=132, Cost=0.0, Power=0.0
  - bb_fabric | Nodes=20, Links=132, Cost=0.0, Power=0.0
    - t2 | Nodes=4, Links=128, Cost=0.0, Power=0.0
    - t1 | Nodes=16, Links=132, Cost=0.0, Power=0.0
  - remote | Nodes=1, Links=4, Cost=0.0, Power=0.0
